<a href="https://colab.research.google.com/github/markNZed/GPT-NeoX-Colab/blob/main/notebooks/shakespeare_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datetime import datetime
current_time = datetime.now()
print("Current Date and Time:", current_time)


Current Date and Time: 2024-10-29 11:14:51.094627


In [2]:
# Mount Google Drive
SAVE_TO_DRIVE = False
if SAVE_TO_DRIVE:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content
!git clone https://github.com/markNZed/GPT-NeoX-Colab.git
!ls

/content
Cloning into 'GPT-NeoX-Colab'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 142 (delta 75), reused 51 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (142/142), 3.60 MiB | 6.06 MiB/s, done.
Resolving deltas: 100% (75/75), done.
drive  GPT-NeoX-Colab  sample_data


# Cloning GPT-NeoX Repo

In [4]:
%cd /content
!git clone https://github.com/EleutherAI/gpt-neox

/content
Cloning into 'gpt-neox'...
remote: Enumerating objects: 19481, done.
remote: Counting objects: 100% (1684/1684), done.
remote: Compressing objects: 100% (812/812), done.
remote: Total 19481 (delta 1241), reused 1196 (delta 864), pack-reused 17797 (from 1)
Receiving objects: 100% (19481/19481), 113.67 MiB | 23.04 MiB/s, done.
Resolving deltas: 100% (14098/14098), done.


# Installing Dependencies

In [ ]:
%cd /content/gpt-neox
!pip install -r ./requirements/requirements.txt
# Currently deepsped rom GTP-NeoX is not compatibel with logging in torch >= 2.4
!pip install torch==2.3

/content/gpt-neox
  Cloning https://github.com/EleutherAI/DeeperSpeed.git (to revision 02e2ebf7dee6aaab3d89094ed470a4609763c742) to /tmp/pip-install-gwehptw4/deepspeed_2f33b10be684433c8a42edcf96180467
  Running command git clone --filter=blob:none --quiet https://github.com/EleutherAI/DeeperSpeed.git /tmp/pip-install-gwehptw4/deepspeed_2f33b10be684433c8a42edcf96180467
  Running command git rev-parse -q --verify 'sha^02e2ebf7dee6aaab3d89094ed470a4609763c742'
  Running command git fetch -q https://github.com/EleutherAI/DeeperSpeed.git 02e2ebf7dee6aaab3d89094ed470a4609763c742
  Running command git checkout -q 02e2ebf7dee6aaab3d89094ed470a4609763c742
  Resolved https://github.com/EleutherAI/DeeperSpeed.git to commit 02e2ebf7dee6aaab3d89094ed470a4609763c742
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/EleutherAI/lm_dataformat.git (to revision 4eec05349977071bf67fc072290b95e31c8dd836) to /tmp/pip-install-gw

# Preparing Custom Dataset

In [ ]:
%cd /content/gpt-neox
!mkdir -p data

import json

# Generate a list of dictionaries
lines = []
with open("/content/GPT-NeoX-Colab/data/shakespeare.txt", encoding="utf8") as f:
    for line in f.read().splitlines():
        if line:
            lines.append({"text": line})

# Convert to a list of JSON strings
json_lines = [json.dumps(l) for l in lines]

# Join lines and save to .jsonl file
json_data = '\n'.join(json_lines)
with open('/content/gpt-neox/data/tinyshakespeare.jsonl', 'w') as f:
    f.write(json_data)

In [ ]:
# This is using enwik8 not our data
%cd /content/gpt-neox
!python prepare_data.py -d ./tokenizer

# TinyShakespeare Dataset

# Tokenizing Dataset

In [ ]:
%cd /content/gpt-neox
!mkdir -p processed_data
!python tools/datasets/preprocess_data.py \
    --input ./data/tinyshakespeare.jsonl \
    --output-prefix ./processed_data/tinyshakespeare \
    --vocab-file ./tokenizer/gpt2-vocab.json \
    --merge-file ./tokenizer/gpt2-merges.txt \
    --dataset-impl mmap \
    --tokenizer-type GPT2BPETokenizer \
    --append-eod


In [ ]:
%load_ext tensorboard

In [ ]:
# Need to remove evrything in checkpoints and tensorboard dir for a fresh run
%tensorboard --logdir tensorboard
# See logs dir for logs
!python ./deepy.py train.py -d configs /content/GPT-NeoX-Colab/configs/shakespeare.yml /content/GPT-NeoX-Colab/configs/shakespeare_deepy.yml > /dev/null

# Inference
The output of the model will be stored in `sample_output.txt` file

In [ ]:
!python ./deepy.py generate.py -d configs /content/GPT-NeoX-Colab/configs/shakespeare.yml /content/GPT-NeoX-Colab/configs/shakespeare_deepy.yml text_generation.yml
!cat sample_output.txt

# Storing Repo with Checkpoints in Google Drive


In [ ]:
if SAVE_TO_DRIVE:

  import shutil
  import os

  # Define source and destination base paths
  source_folder = '/content/gpt-neox'
  destination_folder = '/content/drive/MyDrive/gpt-neox'

  # Function to add version suffix if destination folder exists
  def get_versioned_folder_path(base_path):
      version = 1
      new_path = base_path
      while os.path.exists(new_path):
          new_path = f"{base_path}_v{version}"
          version += 1
      return new_path

  # Get the versioned destination folder path
  destination_folder_versioned = get_versioned_folder_path(destination_folder)

  # Copy the folder
  shutil.copytree(source_folder, destination_folder_versioned)

  print(f"Folder copied successfully to Google Drive as '{destination_folder_versioned}'!")


